<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/analysis_04_%EB%B0%B0%EA%B8%89%EC%82%AC%EB%B9%84%EC%A4%91%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **기본 세팅** 
- 깃허브 클론 
- 드라이브 마운트 
- 파일 임포트

In [1]:
!git clone https://github.com/yujin-jo/mini-project-team1.git

Cloning into 'mini-project-team1'...
remote: Enumerating objects: 567, done.
remote: Counting objects: 100% (567/567), done.
remote: Compressing objects: 100% (531/531), done.
remote: Total 567 (delta 160), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (567/567), 731.53 KiB | 5.30 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

# I. 가설 설정과 분석 배경

1. 가설  :  2011년 ~ 2020년까지 국내 영화 배급사에서 CJ E&M의 비중은 연도 별로 감소했는가? 
2. 가설을 선택한 이유 : 
    - 기존에는 국내 영화의 경우 CJ E&M이 독점했왔음. 
    - 다른 신생 배급사들은 없었을까 궁금했음 
    -  기존에 있던 영세 배급사들은 성장하거나 망했을까 궁금했고, 여전히 CJ E&M이 영화 산업을 지배하고 있는지 확인하고 싶었음 

# II. 분석 방향 
1. 활용 데이터 
  - 영화 개봉연도 : 연도 별로 영화 수 집계 필요 
  - 영화 배급사 : 영화 별 배급사 정보 (KOBIS 참고) 
  - 영화 수 : 연도 별, 배급사 별로 집계한 영화 수

2. 분석 방법론 
  - 시계열 분석 활용 (2011 ~ 2020년 영화 정보) 
  - 영화 배급사의 종류가 연도 별로 어떻게 달라졌는지 확인  (중소 배급사 등 배급사가 늘었는지 확인하기 위함)
  - 개봉 연도와 제작사 별로 영화 편수를 카운트한다. 
  - 연도 별로 제작사 별 비중을 계산하여 추이를 확인한다. 

3. 시각화 
  - 비율 바 차트



# III. 분석 과정  